### Entity Extraction and biotagging in the healthcare queries in indic language

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_english']
y_test=df3['Manual_Intent']
X_test=df3['question_english']

In [ ]:
#df1=df[['question_english','disease_english','drug_english','treatment_english']]
#df2=df3[['question_english','disease_english','drug_english','treatment_english']]
#print(df2.columns)

In [ ]:
#df1=df1.dropna(subset=['disease_english','drug_english','treatment_english'],how='all')
#df2=df2.dropna(subset=['disease_english','drug_english','treatment_english'],how='all')

In [ ]:
#df1['intent'] = df1[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
#df2['intent'] = df2[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)

In [ ]:
!pip install fuzzywuzzy python-Levenshtein deep-translator

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=60,
    output_dir="/kaggle/working",
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_steps=600,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


In [ ]:
webmd_train=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
webmd_test=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
# extract only reuired english columns
#df1=webmd_train[['question_english','disease_english','drug_english','treatment_english']]
#df2=webmd_test[['question_english','disease_english','drug_english','treatment_english']]
#df1['intent'] = df1[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
#df2['intent'] = df2[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
y_train=webmd_train['Manual_Intent']
X_train=webmd_train['question_english']
y_test=webmd_test['Manual_Intent']
X_test=webmd_test['question_english']

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=90,
    output_dir="/kaggle/working",
    num_train_epochs=2,
    load_best_model_at_end=True,
    save_steps=900,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)
